In [ ]:
let parseGraph (lines: string seq) = 
  lines
  |> Seq.map (fun s -> s.Split('-'))
  |> Seq.collect (fun [|x;y|] -> [(x,y); (y,x)])
  |> Seq.groupBy fst
  |> Map
  |> Map.map (fun k vs -> vs |> Seq.map snd |> Seq.toList |> List.sort)

let example = 
  @"start-A
start-b
A-c
A-b
b-d
A-end
b-end".Split('\n') |> parseGraph

let example2 = 
  @"dc-end
HN-start
start-kj
dc-start
dc-HN
LN-dc
HN-end
kj-sa
kj-HN
kj-dc".Split('\n') |> parseGraph

let example3 = 
  @"fs-end
he-DX
fs-he
start-DX
pj-DX
end-zg
zg-sl
zg-pj
pj-he
RW-he
fs-DX
pj-RW
zg-RW
start-pj
he-WI
zg-he
pj-fs
start-RW".Split('\n') |> parseGraph

let input = File.ReadAllLines "input.txt" |> parseGraph

In [ ]:
let (|Small|Big|Start|End|) = function
  | "start" -> Start
  | "end" -> End
  | s -> if Char.IsUpper s[0] then Big s else Small s

let rec paths graph = function
  | End :: _ as path -> [path |> List.rev]
  | Start :: _ :: _ -> []
  | Small cave :: rest when List.contains cave rest -> []
  | cave :: _ as path -> Map.find cave graph |> List.map (fun x -> x :: path) |> List.collect (paths graph)

paths input ["start"] |> Seq.length

4885

In [ ]:
let rec paths2 graph = function
  | End :: _ as path -> [path |> List.rev]
  | Start :: _ :: _ -> []
  | cave :: rest as path -> 
    Map.find cave graph 
    |> List.map (fun x -> x :: path)
    |> List.collect ((match cave with Small _ when List.contains cave rest -> paths | _ -> paths2) graph)

paths2 input ["start"] |> Seq.length

117095